<a href="https://colab.research.google.com/github/molinajuandomingo24-max/labo2025v/blob/main/src/ensembles/420_ArbolesAzarosos_TareaHogar2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4. Ensembles de Arboles de Decision

Un arbol de decisión es un modelo débil, el aumento del poder predictivo proviene al ensamblar varios arboles de decisión.
<br> Si promedio n arboles identicos, el resultados es exactamente el mismo que utilizar un solo arbol, necesito PERTURBAR cada arbol para disponer de variablidad

la variabilidad provendrá de estas fuentes:


*   Perturbar el dataset
*   Perturbar el algoritmo del arbol
*   Perturbar el dataset y el algoritmo del arbol al mismo tiempo

Se verán estos tres algoritmos


*   Arboles Azarosos
*   Random Forest
*   Gradient Boosting of Decision Trees

#### 4.01 Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/austral2025-af91/dataset_pequeno.csv"
archivo_destino="/content/datasets/dataset_pequeno.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/dataset_pequeno.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi



---



## 4.02 Arboles Azarosos

Arboles Azarosos es el nombre de un algoritmo trivial (por favor NO confundir con Random Forest)
Qué tipo de perturbaciones se realizan en Arboles Azarosos
* Se perturba el dataset
* No se perturba el algoritmo, es siempre rpart original

Cada  arbolito de  Arboles Azarosos se entrena sobre un dataset perturbado,  que tiene exactamente la misma cantidad de registros pero solo un subconjunto de los atributos (campos)  del dataset, tomados al azar, de los originales.
<br> En esta primera corrida, se construira cada arbol en un dataset utilizando el 50% de los campos

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Thu Oct 23 10:20:26 PM 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,797754,42.7,1852905,99.0,1837440,98.2
Vcells,1581628,12.1,169264281,1291.4,211580351,1614.3


In [ ]:
# cargo las librerias que necesito
require("data.table")
require("rpart")

Aqui debe cargar SU semilla primigenia

In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- 209353

# parametros  arbol
# entreno cada arbol con solo 50% de las variables variables
#  por ahora, es fijo
PARAM$feature_fraction <- 0.5

PARAM$rpart$cp <- -0.5
PARAM$rpart$minsplit <- 50 #c(50, 40, 30, 20)
PARAM$rpart$minbucket <- 20
PARAM$rpart$maxdepth <- 6 #c(6, 8, 10)

# voy a generar 32 arboles,
#  a mas arboles mas tiempo de proceso y MEJOR MODELO,
#  pero ganancias marginales
PARAM$num_trees_max <- 32

In [ ]:
# carpeta de trabajo
setwd("/content/buckets/b1/exp")
experimento <- "exp4021_32arb"
dir.create(experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/dataset_pequeno.csv")

In [ ]:
# defino los dataset de entrenamiento y aplicacion
dtrain <- dataset[foto_mes == 202107]
dfuture <- dataset[foto_mes == 202109]

# arreglo clase_ternaria por algun distraido ""
dfuture[, clase_ternaria := NA ]

In [ ]:
# Establezco cuales son los campos que puedo usar para la prediccion
# el copy() es por la Lazy Evaluation
campos_buenos <- copy(setdiff(colnames(dtrain), c("clase_ternaria")))

In [ ]:
# que tamanos de ensemble grabo a disco
grabar <- c(1, 2, 4, 8, 16, 32)

In [ ]:
tb_prediccion <- dfuture[, list(numero_de_cliente)]
# aqui se va acumulando la probabilidad del ensemble
tb_prediccion[, prob_acumulada := 0]

In [ ]:
set.seed(PARAM$semilla_primigenia) # Establezco la semilla aleatoria

In [ ]:
for (arbolito in seq(PARAM$num_trees_max) ) {
  message( arbolito, " ")
  qty_campos_a_utilizar <- as.integer(length(campos_buenos)
    * PARAM$feature_fraction)

  # elijo los campos al azar
  campos_random <- sample(campos_buenos, qty_campos_a_utilizar)

  # paso de un vector a un string con los elementos
  # separados por un signo de "+"
  # este hace falta para la formula
  campos_random <- paste(campos_random, collapse= " + ")

  # armo la formula para rpart
  formulita <- paste0("clase_ternaria ~ ", campos_random)

  # genero el arbol de decision
  modelo <- rpart(formulita,
    data= dtrain,
    xval= 0,
    control= PARAM$rpart
  )

  # aplico el modelo a los datos que no tienen clase
  prediccion <- predict(modelo, dfuture, type= "prob")

  tb_prediccion[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]

  if (arbolito %in% grabar) {
    umbral_corte <- (1 / 40) * arbolito
    tb_prediccion[, Predicted := as.numeric(prob_acumulada > umbral_corte)]

    archivo_kaggle <- paste0(
        "KA420_",
        sprintf("%.3d", arbolito), # para que tenga ceros adelante
        ".csv"
      )

    # grabo el archivo
    fwrite( tb_prediccion[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

    # subida a Kaggle
    comando <- "kaggle competitions submit"
    competencia <- "-c labo-i-2025-virtual-analista-sr"
    arch <- paste( "-f", archivo_kaggle)

    mensaje <- paste0("-m 'cp=", PARAM$rpart$cp, "  minsplit=", PARAM$rpart$minsplit, "  minbucket=", PARAM$rpart$minbucket, " maxdepth=", PARAM$rpart$maxdepth, "'" )
    linea <- paste( comando, competencia, arch, mensaje)
    salida <- system(linea, intern=TRUE)
    cat(salida)
  }
}


1 



Successfully submitted to LaboI 2025 virtual analista sr

2 



Successfully submitted to LaboI 2025 virtual analista sr

3 

4 



Successfully submitted to LaboI 2025 virtual analista sr

5 



In [ ]:
# --- Inicio del Código R ---

# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

# cargo las librerias que necesito
require("data.table")
require("rpart")
require("tools") # Para basename

# --- Parametros Fijos y Optimizables ---
PARAM <- list()
PARAM$semilla_primigenia <- 102191 # Su semilla primigenia

# Parametros para Arboles Azarosos
PARAM$feature_fraction <- 0.5  # Fracción de campos a usar en cada arbol
PARAM$num_trees_optimo <- 32   # Número de árboles fijo para la optimización

# Rangos para la optimización de hiperparámetros
# Los valores se eligen para crear arboles potencialmente 'overfitteros'
# (menor minsplit, mayor maxdepth) para el ensemble.
PARAM$minsplit_values <- c(50, 40, 30, 20)
PARAM$maxdepth_values <- c(6, 8, 10)
PARAM$cp_value <- -1 # cp se mantiene fijo

# --- Seteo de Directorios ---
# Carpeta de trabajo
# setwd("/content/buckets/b1/exp") # Esta línea se asume ejecutada en el script Python
experimento_base <- "exp420_opt_ensemble32"
# Dir.create solo si no existe, si existe se borran archivos viejos para un experimento limpio
dir.create(paste0("/content/buckets/b1/exp/", experimento_base), showWarnings=FALSE)
setwd(paste0("/content/buckets/b1/exp/", experimento_base))

# --- Lectura de Datos ---
# Se asume que el archivo ya fue descargado en el setup de Python
dataset <- fread("/content/datasets/dataset_pequeno.csv")

# Defino los dataset de entrenamiento y aplicacion (futuro)
dtrain <- dataset[foto_mes == 202107]
dfuture <- dataset[foto_mes == 202109]

# Arreglo clase_ternaria por algun distraido ""
dfuture[, clase_ternaria := NA ]

# Establezco cuales son los campos que puedo usar para la prediccion
campos_buenos <- copy(setdiff(colnames(dtrain), c("clase_ternaria")))

# --- Búsqueda de Hiperparámetros ---
for (minsplit_actual in PARAM$minsplit_values) {
  for (maxdepth_actual in PARAM$maxdepth_values) {

    # 1. Definir los parámetros del arbolito para esta iteración
    # Se ajusta minbucket para ser consistente con minsplit (minbucket = minsplit/2)
    PARAM$rpart_current <- list(
      cp = PARAM$cp_value,
      minsplit = minsplit_actual,
      minbucket = as.integer(minsplit_actual / 2),
      maxdepth = maxdepth_actual
    )

    # Crear un subdirectorio para cada combinación de hiperparámetros
    exp_detalle <- paste0("m", minsplit_actual, "_d", maxdepth_actual)
    dir.create(exp_detalle, showWarnings=FALSE)
    setwd(exp_detalle)

    cat("--- Optimizando: minsplit=", minsplit_actual, ", maxdepth=", maxdepth_actual, " ---\n")

    # Inicializar para el ensemble
    tb_prediccion <- dfuture[, list(numero_de_cliente)]
    tb_prediccion[, prob_acumulada := 0]

    # Establezco la semilla aleatoria para reproducibilidad del ensemble
    set.seed(PARAM$semilla_primigenia)

    # Bucle para entrenar el ensemble de 32 arboles
    for (arbolito in seq(PARAM$num_trees_optimo) ) {

      qty_campos_a_utilizar <- as.integer(length(campos_buenos) * PARAM$feature_fraction)

      # Elijo los campos al azar
      # Usar una semilla específica del loop para reproducibilidad de la selección de campos
      set.seed(PARAM$semilla_primigenia + arbolito)
      campos_random <- sample(campos_buenos, qty_campos_a_utilizar)

      # Paso de un vector a un string con los elementos separados por un signo de "+"
      campos_random <- paste(campos_random, collapse= " + ")

      # Armo la formula para rpart
      formulita <- paste0("clase_ternaria ~ ", campos_random)

      # Genero el arbol de decision
      modelo <- rpart(formulita,
        data= dtrain,
        xval= 0,
        control= PARAM$rpart_current
      )

      # Aplico el modelo a los datos que no tienen clase
      prediccion <- predict(modelo, dfuture, type= "prob")

      # Acumulo la probabilidad
      tb_prediccion[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]

      message(arbolito, " ")
    }

    # 2. Generar el archivo final con el ensemble de 32 árboles

    # El umbral de corte se calcula para un ensemble de N=32 arboles
    # En el original es (1 / 40) * arbolito, para 32 arboles es (1/40) * 32 = 0.8
    # Se utilizará el promedio de probabilidad, y el umbral de corte del archivo original
    # (que es el corte de la probabilidad promedio para Ganancia, tipicamente 1/40 ~ 0.025)

    # Se usará un umbral de corte simplificado (por ejemplo, 1/40) sobre la probabilidad promedio.
    # El ensemble finaliza en el arbolito 32.
    umbral_corte_promedio <- (1 / 40)

    # Se calcula la probabilidad promedio
    tb_prediccion[, prob_promedio := prob_acumulada / PARAM$num_trees_optimo]

    # Clasificación final
    tb_prediccion[, Predicted := as.numeric(prob_promedio > umbral_corte_promedio)]

    archivo_kaggle <- paste0(
      "KA420_Ensemble32_m", minsplit_actual, "_d", maxdepth_actual, ".csv"
    )

    # Grabo el archivo
    fwrite( tb_prediccion[, list(numero_de_cliente, Predicted)],
      file= archivo_kaggle,
      sep= ","
    )

    # 3. Subida a Kaggle para evaluación de la Ganancia
    comando <- "kaggle competitions submit"
    competencia <- "-c labo-i-2025-virtual-analista-sr"
    arch <- paste( "-f", archivo_kaggle)

    mensaje <- paste0("-m 'minsplit=", minsplit_actual,
                      " minbucket=", PARAM$rpart_current$minbucket,
                      " maxdepth=", maxdepth_actual,
                      " Ensemble de ", PARAM$num_trees_optimo, " Arboles Azarosos'")

    linea <- paste( comando, competencia, arch, mensaje)
    salida <- system(linea, intern=TRUE)
    cat(salida)
    cat("Comando Kaggle (simulado):", linea, "\n")

    # Volver al directorio base
    setwd(paste0("/content/buckets/b1/exp/", experimento_base))
  }
}


format(Sys.time(), "%a %b %d %X %Y")

# --- Fin del Código R ---

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,804614,43.0,1949367,104.2,1949367,104.2
Vcells,1596187,12.2,162557710,1240.3,211580351,1614.3


--- Optimizando: minsplit= 50 , maxdepth= 6  ---


1 

2 

3 

4 

5 

6 

7 

8 

9 

10 

11 

12 

13 

14 

15 

16 

17 

18 

19 

20 

21 

22 

23 

24 

25 

26 

27 

28 

29 

30 

31 

32 



Successfully submitted to LaboI 2025 virtual analista srComando Kaggle (simulado): kaggle competitions submit -c labo-i-2025-virtual-analista-sr -f KA420_Ensemble32_m50_d6.csv -m 'minsplit=50 minbucket=25 maxdepth=6 Ensemble de 32 Arboles Azarosos' 
--- Optimizando: minsplit= 50 , maxdepth= 8  ---


1 

2 

3 

4 

5 

6 

7 

8 

9 

10 

11 

12 

13 

14 

15 

16 

17 

18 

19 

20 

21 

22 

23 

24 

25 

26 

27 

28 

29 

30 

31 

32 



...

1 

2 

3 

4 

5 

6 

7 

8 

9 

10 

11 

12 

13 

14 

15 

16 

17 

18 

19 

20 

21 

22 

23 

24 

25 

26 

27 

28 

29 

30 

31 

32 



Successfully submitted to LaboI 2025 virtual analista srComando Kaggle (simulado): kaggle competitions submit -c labo-i-2025-virtual-analista-sr -f KA420_Ensemble32_m50_d10.csv -m 'minsplit=50 minbucket=25 maxdepth=10 Ensemble de 32 Arboles Azarosos' 
--- Optimizando: minsplit= 40 , maxdepth= 6  ---


1 

2 

3 

4 

5 

6 

7 

8 

9 

10 

11 

12 

13 

14 

15 

16 

17 

18 

19 

20 

21 

22 

23 

24 

25 

26 

27 

28 

29 

30 

31 

32 



Successfully submitted to LaboI 2025 virtual analista srComando Kaggle (simulado): kaggle competitions submit -c labo-i-2025-virtual-analista-sr -f KA420_Ensemble32_m40_d6.csv -m 'minsplit=40 minbucket=20 maxdepth=6 Ensemble de 32 Arboles Azarosos' 
--- Optimizando: minsplit= 40 , maxdepth= 8  ---


1 

2 

3 

4 

5 

6 

7 

8 

9 

10 

11 

12 

13 

14 

15 

16 

17 

18 

19 

20 

21 

22 

23 

24 

25 

26 

27 

28 

29 

30 

31 



In [1]:
# =========================================================================
# 1. INICIALIZACION Y PARAMETROS
# =========================================================================

# limpio la memoria
rm(list=ls(all.names=TRUE))
gc(full=TRUE, verbose=FALSE)

# cargo las librerias que necesito
require("data.table")
require("rpart")
require("parallel")
if( ! require("rlist") ) install.packages("rlist")
require("rlist")
if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")
if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")


# Defino los parámetros Fijos del ENSEMBLE y de la BO
PARAM <- list()

PARAM$semilla_primigenia <- 209353
PARAM$experimento <- "HT_BAYESIAN_ENSEMBLE32"
PARAM$BO_iter <- 3 # Cantidad de iteraciones de la Bayesian Optimization

# Parametros FIJOS del Ensemble (Árboles Azarosos)
PARAM$feature_fraction <- 0.5  # 50% de las variables por arbol
PARAM$num_trees_optimo <- 32   # Cantidad de árboles por ensemble

# Parametros Optimizables (Hyperparameter Space)
PARAM$hs <- makeParamSet(
    makeNumericParam("cp", lower= -1, upper= 0.1),
    makeIntegerParam("minsplit", lower= 1L, upper= 8000L),
    makeIntegerParam("minbucket", lower= 1L, upper= 4000L),
    makeIntegerParam("maxdepth", lower= 3L, upper= 20L),
    forbidden= quote(minbucket > 0.5 * minsplit)
)


# =========================================================================
# Seteo de Directorios y Carga de Datos
# =========================================================================
setwd("/content/buckets/b1/exp")
dir.create(PARAM$experimento, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", PARAM$experimento ))

dataset <- fread("/content/datasets/dataset_pequeno.csv")

# =========================================================================
# 2. FUNCIONES BASE (Particionar)
# =========================================================================

particionar <- function(data, division, agrupa = "", campo = "fold",
                        start = 1, seed = NA) {
  if (!is.na(seed)) set.seed(seed)

  bloque <- unlist(mapply(
    function(x, y) {
      rep(y, x)
    }, division,
    seq(from= start, length.out= length(division))
  ))

  data[, (campo) := sample(rep(bloque, ceiling(.N / length(bloque))))[1:.N],
    by= agrupa
  ]
}

# =========================================================================
# 3. FUNCIONES DE EVALUACIÓN (Ensemble y Cross-Validation)
# =========================================================================

# Esta función entrena un ensemble de 32 árboles en los folds de entrenamiento
# y evalúa el ensemble agregado en el fold_test
ArbolSimpleEnsemble <- function(fold_test, param_rpart, param_ensemble) {

  dtrain_fold <- dataset[fold != fold_test]
  dtest_fold <- dataset[fold == fold_test]

  campos_buenos <- setdiff(colnames(dtrain_fold), c("clase_ternaria", "fold"))

  tb_prediccion_test <- dtest_fold[, list(numero_de_cliente)]
  tb_prediccion_test[, prob_acumulada := 0]

  # Fija la semilla para la selección aleatoria de features DENTRO de cada fold
  set.seed(PARAM$semilla_primigenia + fold_test * 1000)

  # Bucle para entrenar el ensemble de 32 arboles
  for (arbolito in seq(param_ensemble$num_trees_optimo)) {
    # Semilla para selección de variables del árbol 'arbolito'
    set.seed(PARAM$semilla_primigenia + fold_test * 1000 + arbolito)

    qty_campos_a_utilizar <- as.integer(length(campos_buenos) * param_ensemble$feature_fraction)
    campos_random <- sample(campos_buenos, qty_campos_a_utilizar)
    campos_random <- paste(campos_random, collapse= " + ")
    formulita <- paste0("clase_ternaria ~ ", campos_random)

    # Genero el arbol de decision usando los hiperparametros de BO (param_rpart)
    modelo <- rpart(formulita, data= dtrain_fold, xval= 0, control= param_rpart)

    # Aplico el modelo al fold de testing
    prediccion <- predict(modelo, dtest_fold, type= "prob")
    tb_prediccion_test[, prob_acumulada := prob_acumulada + prediccion[, "BAJA+2"]]
  }

  # Calculo la Ganancia para el ENSEMBLE AGREGADO
  prob_baja2_promedio <- tb_prediccion_test[, prob_acumulada] / param_ensemble$num_trees_optimo

  ganancia_testing <- dtest_fold[
    prob_baja2_promedio > 1 / 40,
    sum(ifelse(clase_ternaria == "BAJA+2", 117000, -3000))
  ]

  return(ganancia_testing)
}


# Función de Cross-Validation que usa el Ensemble
ArbolesCrossValidationEnsemble <- function(param_rpart, qfolds, pagrupa, semilla, param_ensemble) {

  divi <- rep(1, qfolds)

  # Particiono el dataset en folds
  particionar(dataset, divi, seed= semilla, agrupa= pagrupa)

  # Aplico el ensemble a cada fold en paralelo
  ganancias <- mcmapply(ArbolSimpleEnsemble,
    seq(qfolds),
    MoreArgs= list(param_rpart, param_ensemble), # Pasando los params del ensemble
    SIMPLIFY= FALSE,
    mc.cores= detectCores()
  )

  dataset[, fold := NULL]

  # Promedio las ganancias y las normalizo
  ganancia_promedio <- mean(unlist(ganancias))
  ganancia_promedio_normalizada <- ganancia_promedio * qfolds

  return(ganancia_promedio_normalizada)
}


# Función que mlrMBO llamará
EstimarGananciaEnsemble <- function(x) {
  message(format(Sys.time(), "%a %b %d %X %Y"))
  GLOBAL_iteracion <<- GLOBAL_iteracion + 1

  # Parámetros fijos del ensemble
  param_ensemble_fixed <- list(
    feature_fraction = PARAM$feature_fraction,
    num_trees_optimo = PARAM$num_trees_optimo
  )

  ganancia <- ArbolesCrossValidationEnsemble(
    param_rpart= x, # Hiperparametros del arbol base (de BO)
    qfolds= 5,
    pagrupa= "clase_ternaria",
    semilla= PARAM$semilla_primigenia,
    param_ensemble= param_ensemble_fixed
  )

  # Grabación del log (simplificado para evitar error inicial)
  cat(
    file= "BO_log.txt",
    append= TRUE,
    GLOBAL_iteracion, "\t",
    ganancia, "\t",
    x$cp, "\t",
    x$minsplit, "\t",
    x$minbucket, "\t",
    x$maxdepth, "\n"
  )

  return(ganancia)
}

# =========================================================================
# 4. CONFIGURACION Y EJECUCION DE MBO
# =========================================================================

archivo_log <- "BO_log.txt"
archivo_BO <- "bayesian.RDATA"

# Leo el log si existe (simplificado)
GLOBAL_iteracion <- 0
if (file.exists(archivo_log)) {
  tabla_log <- fread(archivo_log)
  GLOBAL_iteracion <- nrow(tabla_log)
}


funcion_optimizar <- EstimarGananciaEnsemble

configureMlr(show.learner.output= FALSE)

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar,
  minimize= FALSE, # MAXIMIZAR la ganancia
  noisy= TRUE,
  par.set= PARAM$hs, # Parámetros del árbol base
  has.simple.signature= FALSE
)

ctrl <- makeMBOControl(
  save.on.disk.at.time= 600,
  save.file.path= archivo_BO
)

ctrl <- setMBOControlTermination(ctrl, iters= PARAM$BO_iter)
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

surr.km <- makeLearner("regr.km",
  predict.type= "se",
  covtype= "matern3_2", control= list(trace= TRUE)
)


# Inicio la optimización bayesiana
if (!file.exists(archivo_BO)) {
  bayesiana_salida <- mbo(
    fun= obj.fun,
    learner= surr.km,
    control= ctrl
  )
} else {
  bayesiana_salida <- mboContinue(archivo_BO)
}


# =========================================================================
# 5. RESULTADOS
# =========================================================================

# Almaceno y muestro los mejores hiperparametros
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
setorder(tb_bayesiana, -y)

# El log se graba dentro de EstimarGananciaEnsemble. Se puede grabar aquí también
# para asegurar el formato final.
fwrite( tb_bayesiana,
  file= "BO_log_final.txt",
  sep= "\t"
)

cat("\n--- Mejores Hiperparametros del Arbol Base para el Ensemble (32 Árboles) ---\n")
print(tb_bayesiana[
  1, # El primero es el de mejor ganancia ('y')
  list(cp, minsplit, minbucket, maxdepth)
])

# --- Fin del Código R ---

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658983,35.2,1454460,77.7,1454460,77.7
Vcells,1224507,9.4,8388608,64.0,1975128,15.1


Loading required package: data.table

Loading required package: rpart

Loading required package: parallel

Loading required package: rlist

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rlist’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘XML’


Loading required package: rlist

Loading required package: DiceKriging

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘DiceKriging’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DiceKriging

Loading required package: mlrMBO

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘mlrMBO’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



ERROR: Error in setwd("/content/buckets/b1/exp"): cannot change working directory


In [ ]:
format(Sys.time(), "%a %b %d %X %Y")



---

